In [ ]:
import os
import cv2
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib 
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import skimage
import glob

# Root directory of the project
ROOT_DIR = os.getcwd()

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize


from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

import custom 

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

In [ ]:
config = custom.CustomConfig() 
custom_DIR = os.path.join(ROOT_DIR, "CustomImages") #your dataset directory

In [ ]:
# Override the training configurations with a few
# changes for inferencing.
# Here, two parameters we should change
# 1. backbone: resnet 50, resnet101-- from config.py
# 2. Detection_Min_Confidence: 0.5/0.9,... ... --from config.py and custom.py
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

In [ ]:
# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"

In [ ]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
# Load validation dataset
dataset = custom.CustomDataset() 
dataset.load_custom(custom_DIR, "val") # directory of your validation dataset

# Must call before using the dataset
dataset.prepare()

print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

In [ ]:
# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)

In [ ]:
#define the path to your trained weights.
#      after you train your network using training.ipynb, you will get trained weights inside "logs" folder
custom_WEIGHTS_PATH = '...path/to/your/trained_weights/....h5'

# Load weights
print("Loading weights ", custom_WEIGHTS_PATH)
model.load_weights(custom_WEIGHTS_PATH, by_name=True)

### Inferencing

In [ ]:
#-------------------------------------------------------------
# Test results will be saved in the following directory
testresult = '...path/to/directory/...' #set a path where you want to save the resulted images

# Images from the following directory will be tested
test_dir ='...path/to/test/directory...' #set the path of your test images, which you want to detect and segment

# Creating a list with the names of all the test images from the test_dir
testfiles= os.listdir(test_dir)

#looping through all images in the test_dir
start = time.time()
for i in range(len(testfiles)):
    filename= os.path.join(test_dir,testfiles[i])
    image = skimage.io.imread(filename)
    
    #Run Detection
    results = model.detect([image], verbose=1)
    
    #Display results
    ax = get_ax(1)
    r = results[0]
    visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            dataset.class_names, r['scores'], ax=ax,
                            title="Predictions")
    plt.savefig(testresult+testfiles[i])  

end = time.time()
time_taken = (end-start)
print(time_taken)
